In [10]:
from mdml.db import MamlDB
import pandas as pd
from pymatgen.core import Structure
import json

from maml.models import AtomSets

## Use MEGNet v1 for Ef regression

In [3]:
with open("mp_ef_100.json", "r") as f:
    data = json.load(f)
    
structs = [Structure.from_dict(i) for i in data['structure']]
ef = data['Ef']

In [4]:
from maml.describers import MEGNetSite

In [7]:
describer = MEGNetSite(level=1)

In [9]:
features = describer.transform(structs[:80])

In [13]:
model = AtomSets(describer=describer,
                 input_dim=32,  # the site features have dimenions of 32
                 is_embedding=False,
                 compile_metrics=['mae'],
                 loss='mse',
                 is_classification=False
                )

In [31]:
model.fit(features, ef[:80], epochs=10)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 0.0788 - mae: 0.2341
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 0.0712 - mae: 0.2179
Epoch 3/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0482 - mae: 0.1721
Epoch 4/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0499 - mae: 0.1755
Epoch 5/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0415 - mae: 0.1607
Epoch 6/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0294 - mae: 0.1420
Epoch 7/10
3/3 [==============================] - 0s 5ms/step - loss: 0.0341 - mae: 0.1473
Epoch 8/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0263 - mae: 0.1321
Epoch 9/10
3/3 [==============================] - 0s 3ms/step - loss: 0.0208 - mae: 0.1161
Epoch 10/10
3/3 [==============================] - 0s 2ms/step - loss: 0.0206 - mae: 0.1126


In [32]:
test_features = describer.transform(structs[80:])
loss, metric = model.evaluate(test_features, ef[80:], True)

1/1 [==============================] - 0s 16ms/step - loss: 0.1273 - mae: 0.2563


In [18]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 0.882 eV/atom


Similar models can be constructed using V2 and V3 by changing `level` in the `MEGNetSite` describer.

## Use structural vector and MLP for regression

In [21]:
from maml.describers import MEGNetStructure
from maml.models import MLP

In [26]:
describer = MEGNetStructure(mode='final', feature_batch='pandas_concat')
model = MLP(describer=describer,
           input_dim=96,
           compile_metrics=['mae'],
           loss='mse',
           is_classification=False)

In [27]:
features = describer.transform(structs[:80])

In [28]:
model.fit(features, ef[:80], epochs=10)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 2.5584 - mae: 1.3216
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 1.0640 - mae: 0.8274
Epoch 3/10
3/3 [==============================] - 0s 1ms/step - loss: 0.8410 - mae: 0.7687
Epoch 4/10
3/3 [==============================] - 0s 1ms/step - loss: 0.3368 - mae: 0.4620
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.4552 - mae: 0.5853
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.3155 - mae: 0.4806
Epoch 7/10
3/3 [==============================] - 0s 1ms/step - loss: 0.1186 - mae: 0.2746
Epoch 8/10
3/3 [==============================] - 0s 2ms/step - loss: 0.1795 - mae: 0.3274
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.1663 - mae: 0.3117
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 0.0845 - mae: 0.2411


In [29]:
test_features = describer.transform(structs[80:])
loss, metric = model.evaluate(test_features, ef[80:], True)

1/1 [==============================] - 0s 70ms/step - loss: 0.1682 - mae: 0.3181


In [30]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 0.318 eV/atom
